In [1]:
from pandas import DataFrame, read_sql_query, Int32Dtype
from datetime import datetime
from IPython.core.interactiveshell import InteractiveShell
import pymysql
from sqlalchemy import create_engine
from decouple import config 
from dotenv import load_dotenv
InteractiveShell.ast_node_interactivity = "all"

In [2]:
load_dotenv()
# get the environment variables needed
USER= config('USRCaris')
PASSWORD= config('PASSCaris')
HOSTNAME= config('HOSTCaris')
DBNAME= config('DBCaris')

# get the engine to connect and fetch
engine = create_engine(f"mysql+pymysql://{USER}:{PASSWORD}@{HOSTNAME}/{DBNAME}")
query = '''
SELECT 
    dm.id_patient as id_patient,
    d.case_id,
    p.patient_code AS code,
    d.a_non_patisipan_an AS first_name,
    d.b_siyati AS last_name,
    TIMESTAMPDIFF(YEAR,
        d.nan_ki_dat_ou_fet,
        now()) AS age,
    d.nan_ki_dat_ou_fet AS dob,
    d.a1_dat_entvyou_a_ft_jjmmaa_egz_010817 AS interview_date,
    d.e__telefn,
    d.d_adrs AS adress,
    IF(dm.id IS NOT NULL, 'yes', 'no') AS already_in_a_group,
    dm.id_group AS actual_id_group,
    dg.name AS actual_group_name,
    dm.id_parenting_group AS actual_id_parenting_group,
    dpg.name AS actual_parenting_group_name,
    dh.name AS actual_hub,
    ld.name AS actual_departement,
    d.f_komin AS commune,
    d.g_seksyon_kominal AS commune_section,
    d.b1_non_moun_mennen_entvyou_a AS interviewer_firstname,
    d.c1_siyati_moun_ki_f_entvyou_a AS interviewer_lastname,
    d.d1_kad AS interviewer_role,
    d.lot_kad AS interviewer_other_info,
    d.h_kote_entvyou_a_ft AS interview_location,
    d.paran_ou_vivan AS is_your_parent_alive,
    d.i_non_manman AS mothers_name,
    d.j_non_papa AS fathers_name,
    d.k_reskonsab_devan_lalwa AS who_is_your_law_parent,
    d.total,
    d.organisation
FROM
    caris_db.dreams_surveys_data d
        LEFT JOIN
    dream_member dm ON dm.case_id = d.case_id
        LEFT JOIN
    patient p ON p.id = dm.id_patient
        LEFT JOIN
    dream_group dg ON dg.id = dm.id_group
        LEFT JOIN
    dream_group dpg ON dpg.id = dm.id_parenting_group
        LEFT JOIN
    dream_hub dh ON dh.id = dg.id_dream_hub
        LEFT JOIN
    lookup_commune lc ON lc.id = dh.commune
        LEFT JOIN
    lookup_departement ld ON ld.id = lc.departement
'''

sdata = read_sql_query(query,engine,parse_dates=True)
sdata.id_patient = sdata.id_patient.astype(Int32Dtype())

# close the pool of connection
engine.dispose()

True

In [3]:
dtabarre = DataFrame(
    {
        'case_id':[
            '14172120-aa43-4456-82c3-9c38c5da9aaf',
            'dbb335a7-0a50-4e28-8157-32e8a0f5d2d5',
            '62b7012c-235f-4e0f-90e0-3c29d25fc90e',
            '6ab1b8fb-fec0-4867-8b6d-a1adb80c6b2c',
            '218fc9a2-cc8a-4f91-96a8-45a555754090',
            'a80a7e33-0470-49e9-9fda-871f19750f11',
            '3a91ac53-2ebf-49b6-9103-c6dc03611feb',
            'd9cbb68e-0aa4-4dd1-b017-18f6a57d9946',
            '013684fe-9e5c-436e-84c8-2bd12b374af6',
            '381d07d1-bd47-4430-993e-24eec8af792d',
            'bd4729fc-45b9-4da7-8a88-501d3844acd3',
            '41138249-6f19-4004-8e4a-14d954e32a90',
            'd0455ad9-d926-4637-a332-8477fba206f9',
        ]
    }
)

dtabarre.case_id.count()
sdata.case_id.count()

13

67217

In [4]:
twoMore = DataFrame(
    {
        'case_id':[
           '779084fd-e84b-4404-b471-2b20f5f1620c',
           '313b2b6a-17ba-462b-a2c0-dc69fcff5286' 
        ]
    }
)

twoMore.case_id.count()

2

In [5]:
sixMore = DataFrame(
    {
        'case_id':[
           '5765d68a-5362-498e-9af4-77ec4ea803a8',
           'ed3a8101-21cc-44d1-856c-569dfe1951c3',
           '6b99846e-1cf9-42af-be4d-b74a8c3baf51',
           '8be56fdb-81b3-4228-8def-ef7f42699018',
           '58d1a812-2925-467a-af25-5ce0dcf6b4c6',
           '1de990ae-a653-47fc-b7c3-f384b524c614'
        ]
    }
)

sixMore.case_id.count()

6

In [6]:
sixMore_case_id = sdata[sdata.case_id.isin(sixMore.case_id)]
sixMore_case_id

,id_patient,case_id,code,first_name,last_name,age,dob,interview_date,e__telefn,adress,...,interviewer_lastname,interviewer_role,interviewer_other_info,interview_location,is_your_parent_alive,mothers_name,fathers_name,who_is_your_law_parent,total,organisation
7730,83566,1de990ae-a653-47fc-b7c3-f384b524c614,PAP/DRMS/030158004,Rosemina,Lafortune,13.0,2008-09-13,2020-08-03,48316534,Caradeux,...,Phaëth,mentor,None,Village de la grâce,2.0,Mireille Hippolite,,Mr Sygiene,26.0,CARIS
22811,108475,5765d68a-5362-498e-9af4-77ec4ea803a8,PAP/DRMS/030226306,Medjine,St Jules,13.0,2008-08-09,2020-11-12,34208744,Carradeux,...,Cazeau,mentor,None,Carradeux,0.0,Pierre Louis marie Venise,St Jules Benissin,manman,16.0,CARIS
23216,83585,58d1a812-2925-467a-af25-5ce0dcf6b4c6,PAP/DRMS/031158103,sophonia,yliodain,18.0,2003-11-29,2020-08-03,48316534,carradeux,...,phaeth,mentor,None,village de la grace,2.0,sigeurne merilan,,sigeurne merilan,28.0,CARIS
28259,<NA>,6b99846e-1cf9-42af-be4d-b74a8c3baf51,None,Saica,Milien,11.0,2010-05-17,2020-07-11,36309186,Caradeux,...,Phaeth,mentor,None,,2.0,Françoise,,,12.0,CARIS
36764,130041,8be56fdb-81b3-4228-8def-ef7f42699018,PAP/DRMS/215355607,Fadouchard,Clermont,11.0,2010-06-05,2021-11-29,,tabarre,...,Accy,mentor,None,tabarre,0.0,,Clermont,,26.0,CARIS
62248,87358,ed3a8101-21cc-44d1-856c-569dfe1951c3,PAP/DRMS/097161062,Shelda,Simon,15.0,2006-11-22,2020-02-24,36032520,Refugé,...,Cazeau,mentor,None,Caradeux,0.0,Florise,Ernso,Florise,15.0,CARIS


In [7]:
twoMore_case_id = sdata[sdata.case_id.isin(twoMore.case_id)]
twoMore_case_id

,id_patient,case_id,code,first_name,last_name,age,dob,interview_date,e__telefn,adress,...,interviewer_lastname,interviewer_role,interviewer_other_info,interview_location,is_your_parent_alive,mothers_name,fathers_name,who_is_your_law_parent,total,organisation
12703,131538,313b2b6a-17ba-462b-a2c0-dc69fcff5286,PAP/DRMS/215366208,Nahama,Michel,12.0,2009-12-10,2020-10-03,,,...,Phaeth,mentor,None,,2.0,,,,26.0,CARIS
31388,131545,779084fd-e84b-4404-b471-2b20f5f1620c,PAP/DRMS/215366215,Loudjina,Simeon,12.0,2010-03-21,2021-11-20,48940833,village eden,...,Phaeth,mentor,None,village eden,2.0,Simeon,,,32.0,CARIS


In [8]:
found_case_id = sdata[sdata.case_id.isin(dtabarre.case_id)]
found_case_id

,id_patient,case_id,code,first_name,last_name,age,dob,interview_date,e__telefn,adress,...,interviewer_lastname,interviewer_role,interviewer_other_info,interview_location,is_your_parent_alive,mothers_name,fathers_name,who_is_your_law_parent,total,organisation
324,128298,013684fe-9e5c-436e-84c8-2bd12b374af6,PAP/DRMS/031342801,Shelda,Joseph,16.0,2006-02-16,2021-10-22,31113496,carradeux,...,Louis Jean,mentor,None,village de la Grace,0.0,Sanon Rachel,Joseph Jean Marck,None,32.0,Caris
5241,76977,14172120-aa43-4456-82c3-9c38c5da9aaf,PAP/DRMS/097081306,Kerwine,MEANCE,12.0,2009-10-28,2020-07-14,39947568,carradeux,...,Valestil,mentor,None,Ketia,0.0,Alexandre Meance,Donese,,20.0,CARIS
8662,122958,218fc9a2-cc8a-4f91-96a8-45a555754090,PAP/DRMS/030296503,Guerdie,Merzy,16.0,2005-09-04,2021-08-31,34782560,carradeux,...,Louis Jean,mentor,None,village de la grace,4.0,None,None,grand soeur,19.0,CARIS
14562,130037,381d07d1-bd47-4430-993e-24eec8af792d,PAP/DRMS/215355603,Rachida,Simeon,10.0,2011-07-15,2021-11-26,,tabarre,...,Accy,mentor,None,Tabarre,0.0,,Simeon,,26.0,CARIS
15172,130038,3a91ac53-2ebf-49b6-9103-c6dc03611feb,PAP/DRMS/215355604,Syndia Santa,Surin,14.0,2008-04-05,2021-11-26,,tabarre,...,Accy,mentor,None,tabarre,0.0,,Surin,,26.0,CARIS
16930,84644,41138249-6f19-4004-8e4a-14d954e32a90,PAP/DRMS/031163216,marie Milanne,remy,23.0,1998-05-22,2020-08-15,,,...,Phaëth,mentor,None,,2.0,,Jean felix,,22.0,CARIS
25897,87297,62b7012c-235f-4e0f-90e0-3c29d25fc90e,PAP/DRMS/097161002,Mirlande,Tanis,21.0,2000-11-28,2020-08-15,34262632,Carradeux (Camp Toto),...,Phaëth,mentor,None,Village de la grâce,2.0,Manette,Eldieu,Guerline Mondesir,45.0,CARIS
28027,128301,6ab1b8fb-fec0-4867-8b6d-a1adb80c6b2c,PAP/DRMS/031342804,Fritzlene,Fevrin,17.0,2005-03-03,2021-10-23,48628298,village de la grace,...,Cazeau,mentor,None,carradeux,0.0,Maculene,Frisnel,maman,26.0,Caris
44236,122972,a80a7e33-0470-49e9-9fda-871f19750f11,PAP/DRMS/030296517,Claudiana,Mezy,14.0,2007-10-22,2021-08-06,44245927,carradeux,...,Louis Jean,mentor,None,village de la grace,0.0,Ruthlene,Claude Jean,maman,18.0,CARIS
49789,122977,bd4729fc-45b9-4da7-8a88-501d3844acd3,PAP/DRMS/030296522,Noradine,Normil,11.0,2011-02-28,2021-05-01,36309186,Carradeux,...,Phaeth,mentor,None,village de la grâce,2.0,Françoise Devoline,Noramix Normil,None,25.0,Caris


In [9]:
found_case_id.to_excel('./tabarre_found.xlsx',na_rep='',index=False)
twoMore_case_id.to_excel('./twoMore_tabarre_found.xlsx',na_rep='',index=False)
sixMore_case_id.to_excel('./twoMore_tabarre_found.xlsx',na_rep='',index=False)